In [ ]:
#get osparc imports
import osparc
from osparc.models import File, Solver, Job, JobStatus, JobInputs, JobOutputs
from osparc.api import FilesApi, SolversApi, UsersApi
import time #progress

#### The API key and API secret are obatained by logging into OSPARC and generating a new key

In [ ]:
#Adding the API key secret pair to environmental variables; replace "secret" with your API key/secret
os.environ["OSPARC_API_KEY"] = "Secret"
os.environ["OSPARC_API_SECRET"] = "Secret"

In [ ]:
cfg = osparc.Configuration(
    username=os.environ["OSPARC_API_KEY"],
    password=os.environ["OSPARC_API_SECRET"],
)
print(cfg.host)
#Configuration used to create an instance of the API client - for talking to osparc


In [ ]:
#priting out user information - not required
with osparc.ApiClient(cfg) as api_client:

    users_api = UsersApi(api_client)

    profile = users_api.get_my_profile()
    print(profile)

In [ ]:
with osparc.ApiClient(cfg) as api_client:

    files_api = FilesApi(api_client)
    #Uploading files stored locally with correct order of inputs
    input_file_1: File = files_api.upload_file(file="C-FINE.json")
    input_file_2: File = files_api.upload_file(file="sub-57_sam-1.xml")
    input_file_3: File = files_api.upload_file(file = "sub-57_sam-1.json" )

    solvers_api = SolversApi(api_client)
    #Choose the correct name and version
    solver: Solver = solvers_api.get_solver_release(
        "simcore/services/comp/nerve-mesh", "1.0.1"
    )

    #Set up the JOB, input names must match solver   
    job: Job = solvers_api.create_job(
        solver.id,
        solver.version,
        JobInputs(
            {
                "input_3": input_file_3,
                "input_2": input_file_2,
                "input_1": input_file_1,
            }
        ),
    )
        
        
    #Status of the solver; (something wrong here; fine until here)
    status: JobStatus = solvers_api.start_job(solver.id, solver.version, job.id)
    while not status.stopped_at:
        time.sleep(3)
        status = solvers_api.inspect_job(solver.id, solver.version, job.id)
        print("Solver progress", f"{status.progress}/100", flush=True)

    #retrieve outputs and print results
    outputs: JobOutputs = solvers_api.get_job_outputs(solver.id, solver.version, job.id)

    print(f"Job {outputs.job_id} got these results:")
    for output_name, result in outputs.results.items():
        print(output_name, "=", result)

    results_file: File = outputs.results["output_1"]
    download_path: str = files_api.download_file(file_id=results_file.id)
    print(Path(download_path).read_text())
